$$
\begin{array}{l}
\textbf{Project Proposal} \\\\

\textbf{Group Information:} \\\\
\quad \text{Team Members: Gilad Bregman, Yaniv Arbel, Bar Maman} \\\\

\textbf{Topic Selection:} \\\\
\quad \text{Chosen Topic: Voice Reconstruction} \\\\

\textbf{Project Scope and Objectives} \\\\

\textbf{Scope:} \\\\
\quad \text{The project focuses on decomposing an audio recording into its fundamental frequencies. We aim to explore how audio signals can be both disassembled and reassembled using these frequency components. Every audio piece can be described as a mathematical function, which can be expressed as a linear combination of sine and cosine functions. We will approach each function as being spanned by vectors of sine and cosine.} \\\\

\textbf{Objectives:} \\\\
\quad \text{1. Understand and apply linear algebra principles in audio signal decomposition.} \\\\
\quad \text{2. Explore the Discrete Fourier Transform (DFT) as a tool for frequency decomposition.} \\\\
\quad \text{3. Reconstruct original audio from frequency components to assess DFT accuracy and efficiency.} \\\\

\textbf{Research Approach:} \\\\
\quad \text{Our group will begin by collecting diverse audio data. We will apply the Discrete Fourier Transform (DFT) to convert audio segments from the time domain} \\ \text{into the frequency domain, allowing us to examine frequency components and understand their roles in the overall audio signal. Our findings will emphasize how these frequencies characterize the audio.} \\\\

\textbf{Mathematical Methods:} \\\\
\quad \text{• Linear transformations and matrix multiplications.} \\\\
\quad \text{• Representation of sine and cosine spaces as vectors for expressing audio signals.} \\\\
\quad \text{• Series and approximations: exponential series, sine/cosine series, Fourier series.} \\\\
\quad \text{• Complex numbers.} \\\\
\quad \text{• Vector dot products.} \\\\

\textbf{Resources and Tools} \\\\

\textbf{Data Sources:} \\\\
\quad \text{The audio data will be sourced from recordings we create, covering a variety of sounds.} \\\\

\textbf{Tools and Software:} \\\\
\quad \text{We will use Python with libraries like NumPy for numerical calculations, Matplotlib for visualization, and SciPy for DFT functions to conduct our analyses.} \\\\

\textbf{Timeline} \\\\

\textbf{Milestones:} \\\\
\quad \text{• Nov 18-24: Study required mathematical concepts.} \\\\
\quad \text{• Nov 25-Dec 1: Design the algorithm and code in Python.} \\\\
\quad \text{• Dec 2-8: Record sounds and test the algorithm on various cases.} \\\\

\textbf{Potential Challenges and Solutions} \\\\

\textbf{Challenges:} \\\\
\quad \text{1. Studying complex algorithms and math concepts we're less familiar with.} \\\\
\quad \text{2. Working with complex numbers, which are new to us.} \\\\
\quad \text{3. Dealing with noisy, real-world audio data that complicates analysis.} \\\\

\textbf{Solutions:} \\\\
\quad \text{• We will learn new mathematical concepts and the DFT algorithm thoroughly.} \\\\
\quad \text{• We'll focus on applying linear algebra to non-complex numbers initially.} \\\\
\quad \text{• We will record clear audio and test various edge cases.} \\\\

\textbf{Signatures:} \\\\
\quad \text{Yaniv Arbel} \\\\
\quad \text{Bar Maman} \\\\
\quad \text{Gilad Bregman}
\end{array}
$$
